# Reinforcement Learning
- Following [this](https://www.youtube.com/watch?v=Mut_u40Sqz4) tutorial

## Import dependencies

In [1]:
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 223, in _main
    status = self.run(options, args)
  File "/usr/lib/python3.9/site-packages/pip/_internal/cli/req_command.py", line 180, in wrapper
    return func(self, options, args)
  File "/usr/lib/python3.9/site-packages/pip/_internal/commands/install.py", line 320, in run
    requirement_set = resolver.resolve(
  File "/usr/lib/python3.9/site-packages/pip/_internal/resolution/resolvelib/resolver.py", line 121, in resolve
    self._result = resolver.resolve(
  File "/usr/lib/python3.9/site-packages/resolvelib/resolvers.py", line 454, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/lib/python3.9/site-packages/resolvelib/resolvers.py", line 348, in resolve
    failure_causes = self._attempt_to_pin_criterion(name, criterion)
  File

In [2]:
import gym
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Load Environment

In [3]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

### Test the environment

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: 11.0
Episode: 2, Score: 18.0
Episode: 3, Score: 28.0
Episode: 4, Score: 44.0
Episode: 5, Score: 16.0


## Train a RL Model

In [4]:
log_path = os.path.join("../output", "training", "logs")

In [6]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [7]:
model.learn(total_timesteps=20000)

Logging to ../output/training/logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 923  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 613         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009054661 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00526    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.74        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0179     |
|    value_loss           | 52.8        |
-----------------------------------

## Save and Reload Model

In [8]:
ppo_path = os.path.join("../output", "training", "models", "ppo_model_partpole")

In [9]:
model.save(ppo_path)

In [10]:
del model

In [11]:
model = PPO.load(ppo_path, env=env)

## Evaluation

In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

/home/moritz/.local/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

## Use model to predict action

In [47]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

Episode: 1, Score: [10.]
Episode: 2, Score: [9.]
Episode: 3, Score: [10.]
Episode: 4, Score: [10.]
Episode: 5, Score: [8.]


## Viewing logs in Tensorboard

In [1]:
%load_ext tensorboard

In [5]:
%tensorboard --logdir={log_path}

Reusing TensorBoard on port 6006 (pid 24201), started 5:34:25 ago. (Use '!kill 24201' to kill it.)

## Adding Callbacks (auto save)

In [28]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [32]:
save_path = os.path.join("../output", "training", "saved models")

In [33]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=save_path,
    verbose=1)


In [34]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to ../output/training/logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 910  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 593         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008688759 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00871     |
|    learning_rate        | 0.0003      |
|    loss                 | 5.78        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0154     |
|    value_loss           | 48.9        |
-----------------------------------

/home/moritz/.local/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.008018769 |
|    clip_fraction        | 0.0697      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.613      |
|    explained_variance   | 0.333       |
|    learning_rate        | 0.0003      |
|    loss                 | 26.3        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0155     |
|    value_loss           | 62.5        |
-----------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


## Change Policies

In [37]:
net_arch = [dict(
    pi=[128, 128, 128, 128],
    vf=[128, 128, 128, 128])]

In [39]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path,
            policy_kwargs={'net_arch': net_arch})

Using cuda device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to ../output/training/logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 762  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 499         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014213986 |
|    clip_fraction        | 0.217       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00771     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.62        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0229     |
|    value_loss           | 21.2        |
-----------------------------------

/home/moritz/.local/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 200         |
|    mean_reward          | 200         |
| time/                   |             |
|    total timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.012269271 |
|    clip_fraction        | 0.122       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.56       |
|    explained_variance   | 0.321       |
|    learning_rate        | 0.0003      |
|    loss                 | 20.3        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0173     |
|    value_loss           | 49.6        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 400   |
|    iterations      | 5     |
|    time_elapsed    | 25    |


## Using alternative training algorithm (`DQN`)

In [42]:
from stable_baselines3 import DQN

In [43]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [48]:
model.learn(total_timesteps=200000, callback=eval_callback)

Logging to ../output/training/logs/DQN_3
----------------------------------
| rollout/            |          |
|    exploration rate | 0.994    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4702     |
|    time_elapsed     | 0        |
|    total timesteps  | 117      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.989    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 5989     |
|    time_elapsed     | 0        |
|    total timesteps  | 231      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.986    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 5974     |
|    time_elapsed     | 0        |
|    total timesteps  | 286      |
----------------------------------
--------------